In [4]:
import tellurium as te
import matplotlib.pyplot as plt

# function for plotting the phase portraits of a given model
# specify metabolites_list to analyze a specific set of metabolites
def plot_phase_portraits(model, simulation, metabolites_list):
    model_phase_portraits = {}
    used_metabolites = []
    model_metabolites= model.getFloatingSpeciesIds()
    for metabolite in metabolites_list:
        metabolite_simulation_identifier = '[' + metabolite + ']'
        model_phase_portraits[metabolite] = simulation[metabolite_simulation_identifier]
    for i in metabolites_list:
        for j in metabolites_list:
            if i != j and j not in used_metabolites:
                plt.figure()
                plt.title("Comparison between {} and {}".format(i,j))
                plt.xlabel(i)
                plt.ylabel(j)
                plt.plot(model_phase_portraits[i], model_phase_portraits[j])
        used_metabolites.append(i)


In [3]:
asansm_path = 'fbc/iPAE1146_Amino_sugar_and_nucleotide_sugar_metabolism_fbc_squeezed.xml'
pm_path = 'fbc/iPAE1146_Pyrimidine_metabolism_fbc_squeezed.xml'
lb_path = 'fbc/iPAE1146_Lipopolysaccharide_biosynthesis_fbc_squeezed.xml'
combined_model_path = 'fbc/iPAE1146_Combined_Subsystems_fbc_squeezed.xml'

asansm_model = te.loadSBMLModel(asansm_path)
pm_model = te.loadSBMLModel(pm_path)
lb_model = te.loadSBMLModel(lb_path)
combined_model = te.loadSBMLModel(combined_model_path)


In [ ]:
asansm_model.reset()
asansm_simulation = asansm_model.simulate(0,100,1000)
asansm_model.plot(asansm_simulation)

asansm_model.getSteadyStateValues()
# asansm_phase_metabolites = [] # insert the metabolites to be considered in the phase portraits

# plot_phase_portraits(asansm_model, asansm_simulation, asansm_phase_metabolites)

In [3]:
pm_model.reset()
pm_simulation = pm_model.simulate(0,100,1000)
pm_model.plot(pm_simulation)

pm_model.getSteadyStateValues()

# plot_phase_portraits(pm_model, pm_simulation)

In [3]:
lb_model.reset()
lb_simulation = lb_model.simulate(0,100,1000)
lb_model.plot(lb_simulation)

lb_model.getSteadyStateValues()

# plot_phase_portraits(lb_model, lb_simulation)

In [3]:
combined_model.reset()
combined_model_simulation = combined_model.simulate(0,100,1000)
combined_model.plot(combined_model_simulation)

combined_model.getSteadyStateValues()

# plot_phase_portraits(combined_model, combined_model_simulation)